### Example 1a: Running the Full Analysis Pipeline without PWC
You can run the entire analysis pipeline on a results folder by calling the `run_all` function:

In [ ]:
import wizards_staff # import the module

# Path to the example data folder (update this path to your own data directory)
results_folder = './path/to/results' # Folder containing the results of the Lizard_Wizard pipeline
metadata_path = './path/to/metadata.csv' # Path to the metadata file
frate = 30  # Written in Frames Per Second

# Run the full analysis pipeline:
rise_time_df, fwhm_df, frpm_df, mask_metrics_df, silhouette_scores_df = wizards_staff.run_all(results_folder,  # Folder containing the results of the Lizard_Wizard pipeline
                                                                                            metadata_path, # Path to the metadata file
                                                                                            frate=frate, # Frame rate of the video
                                                                                            size_threshold=20000, # Parameter to filter out large noise 
                                                                                            show_plots = True, # Set to True to show the plots inline
                                                                                            save_files = False # Set to True to save the output files
                                                                                            )

### Example 1b: Running the Full Analysis Pipeline with PWC
You can run the entire analysis pipeline including pairwise correlation calculations on a results folder by calling the `run_all` function and specifying a group name which is specified by a column name in your 'metadata_path' CSV file:

In [ ]:
import wizards_staff # import the module

# Path to the example data folder (update this path to your own data directory)
results_folder = './path/to/results' # Folder containing the results of the Lizard_Wizard pipeline
metadata_path = './path/to/metadata.csv' # Path to the metadata file
frate = 30  # Written in Frames Per Second

# Run the full analysis pipeline:
rise_time_df, fwhm_df, frpm_df, mask_metrics_df, silhouette_scores_df = wizards_staff.run_all(results_folder,  # Folder containing the results of the Lizard_Wizard pipeline
                                                                                            metadata_path, # Path to the metadata file
                                                                                            frate=frate, # Frame rate of the video
                                                                                            group_name = 'group', # Column name in the metadata file containing the group information
                                                                                            size_threshold=20000, # Parameter to filter out large noise 
                                                                                            show_plots = True, # Set to True to show the plots inline
                                                                                            save_files = False # Set to True to save the output files
                                                                                            )

### Example 2: Running just the PWC Function
You can run just the PWC pipeline on a results folder by calling the `run_all` function:

In [ ]:
import wizards_staff # import the module

# Path to the example data folder (update this path to your own data directory)
results_folder = './path/to/results' # Folder containing the results of the Lizard_Wizard pipeline
metadata_path = './path/to/metadata.csv' # Path to the metadata file
frate = 30  # Written in Frames Per Second

# Run the full analysis pipeline:


### Example 3: Running just one Function instead of `run_all`
